## Importamos las librerias necesarias

In [12]:
#Importamos pandas, numpy
import numpy as np
import pandas as pd

## Importamos el dataset limpio para cargarlo en un dataframe

In [13]:
#Guardamos el path del archivo en una variable para despues cargarlo en un df

file = 'C:/Users/Ramiro/Desktop/Digital House/Desafios/Desafio 2/dataset limpio.csv'

# Creamos un DataFrame con la info del csv
df = pd.read_csv(file)

### Dejamos solo las columnas necesarias

In [14]:
df = df[['Tipo_Propiedad', 'Barrio','Provincia', 'Precio', 'Precio_ARS', 'Precio_Dolares', 'Metros_Totales',
       'Metros_Cubiertos', 'Precio_m2_Dls', 'Precio_m2', 'Cantidad_Ambientes']]

In [ ]:
df.info()

### Antes de eliminar observaciones, deberiamos chequear la distribucion por Provincias para asegurarnos que no nos queden zonas sin observaciones

In [ ]:
df['Provincia'].value_counts()

Vemos que en el dataset original existen provincias con muy pocas observaciones. Esto generara que el modelo no pueda predecir correctamente, o del todo, esas provincias.

### Nos quedamos solo con las observaciones que no tienen valores nulos

Prodiamos intentar predecir cualquiera de las variables de precio (en dolares o pesos). Para evaluar que tan bien performan los modelos, vamos a intentar predecir tanto el Precio total en pesos (Precio_ARS) y el Precio por metro cuadrado en pesos (Precio_m2)

In [15]:
df_precio = df.drop(['Precio', 'Precio_Dolares','Precio_m2_Dls', 'Precio_m2'], axis = 1)

df_m2 = df.drop(['Precio', 'Precio_Dolares','Precio_m2_Dls', 'Precio_ARS'], axis = 1)

In [16]:
df_precio = df_precio.dropna(axis =0 , how = 'any')
df_m2 = df_m2.dropna(axis =0 , how = 'any')

In [ ]:
#Dataset Precio total
df_precio['Provincia'].value_counts()

In [ ]:
#Dataset Precio m2
df_m2['Provincia'].value_counts()

Efectivamente, notamos que al momento de quedarnos solo con las observaciones con todos los valores completos, perdimos Provincias enteras como Formosa, Catamarca y Jujuy.
Esto va a causar que al momento de querer realizar una prediccion sobre una propiedad de esa provincia, el modelo prediga incorrectamente.
Entendemos que es un error a tener en cuenta. La forma de poder corregirlo seria agregar observaciones al dataset.

## Modelado

Para comenzar el modelado, debemos transformar las variables categoricas en dummies, luego normalizaremos las variables para facilitar la corrida de los distintos modelos.
Vamos a modelar primero tratando predecir el Precio total para todo el pais, y luego el precio por metro cuadrado.

Finalmente, generaremos modelos mas acotados (a ciertas provincias o zonas) buscando una performance mejor de los modelos

In [17]:
#Creamos variables dummies de las variables categoricas

#Provincia
df_precio = pd.concat([df_precio, pd.get_dummies(data = df_precio['Provincia'], prefix= 'prov')], axis = 1)
df_m2 = pd.concat([df_m2, pd.get_dummies(data = df_m2['Provincia'], prefix= 'prov')], axis = 1)

#Barrio
df_precio = pd.concat([df_precio, pd.get_dummies(data = df_precio['Barrio'], prefix= 'barrio')], axis =1)
df_m2 = pd.concat([df_m2, pd.get_dummies(data = df_m2['Barrio'], prefix= 'barrio')], axis = 1)

#Tipo Propiedad
df_precio = pd.concat([df_precio, pd.get_dummies(data = df_precio['Tipo_Propiedad'], prefix= 'tipo_prop')], axis =1)
df_m2 = pd.concat([df_m2, pd.get_dummies(data = df_m2['Tipo_Propiedad'], prefix= 'tipo_prop')], axis = 1)

In [18]:
#Dropeo de las columnas originales
df_precio = df_precio.drop(['Provincia','Barrio','Tipo_Propiedad'], axis = 1)
df_m2 = df_m2.drop(['Provincia','Barrio','Tipo_Propiedad'], axis = 1)

#### Importamos todas las librerias necesarias para el modelado

In [19]:
from sklearn import linear_model
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split

#### Armamos nuestro vector objetivo y nuestra matriz de features

In [20]:
#Separamos nuestra matriz de features

X = df_precio.drop(['Precio_ARS'], axis = 1)
y = df_precio['Precio_ARS']

#### Separamos los datos en sets de testeo y training

In [21]:
#Separamos en train y test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=1)

### Regresion Lineal

In [22]:
#Instanciamos el modelo

lm = linear_model.LinearRegression()

#Entrenamos el modelo
reg_lineal = lm.fit(X_train, y_train)

#Guardamos el score del modelo en una variable para poder comparar
R2_reg_lin_precio = reg_lineal.score(X_test, y_test)

#Prediccion con el set de testeo

print('Score del Regresion Lineal:', reg_lineal.score(X_test, y_test))

Score del Regresion Lineal: 0.568574891331734


#### Como predice el mismo modelo el precio por metro cuadrado?

In [29]:
#Separamos nuestra matriz de features

X_m2 = df_m2.drop(['Precio_m2'], axis = 1)
y_m2 = df_m2['Precio_m2']

In [42]:
#Separamos en train y test
X_train_m2, X_test_m2, y_train_m2, y_test_m2 = train_test_split(X_m2, y_m2, test_size=0.10, random_state=15)

In [43]:
#Instanciamos el modelo

lm_m2 = linear_model.LinearRegression()

#Entrenamos el modelo
reg_lineal_m2 = lm_m2.fit(X_train_m2, y_train_m2)

#Guardamos el score del modelo en una variable para poder comparar
R2_reg_lin_m2 = reg_lineal_m2.score(X_test_m2, y_test_m2)

#Prediccion con el set de testeo

print('Score del Regresion Lineal:', reg_lineal_m2.score(X_test_m2, y_test_m2))

Score del Regresion Lineal: 0.22195752473918462


## Meter un grafico aca comparando los dos scores???

In [56]:
print(R2_reg_lin_precio, ',',R2_reg_lin_m2)

0.568574891331734 , 0.22195752473918462


Vemos que hay una diferencia considerable entre el score del modelo que predice el precio total, que el que predice el precio por metro cuadrado. Tengamos en cuenta que en ninguno de los dos modelos incluimos la otra variable al momento de modelar, ya que entendemos que por ejemplo, el precio por metro cuadrado explicaria casi el total de la variabilidad del Precio Total de la propiedad

El modelo de regresion lineal multiple es demasiado simple para poder obtener un insight lo suficientemente convincente, por lo cual vamos a aplicar dos modelos que incluyen tecnicas de regularizacion: Ridge y Lasso

### Primero debemos normalizar las features.
Las variables dummies no es necesario que sean normalizadas ya que son variables dicotomicas (0 o 1).

Recordemos que nuestros sets de training y testeo quedaron definidos de la siguiente manera:

`X_train, X_test, y_train, y_test`: Para el modelar el Precio_ARS

`X_train_m2, X_test_m2, y_train_m2, y_test_m2`: Para modelar el Precio_m2

In [44]:
#Importamos StandardScaler para normalizar las variables
from sklearn.preprocessing import StandardScaler
scaler_precio = StandardScaler()
X_train = scaler_precio.fit_transform(X_train)

In [47]:
print('media:',np.mean(X_train[:,1]))
print('desvio:',np.std(X_train[:,1]))

media: -2.758597434671737e-17
desvio: 0.9999999999999999


# xq no da 0 y 1 exactamente???

## Regresion Lineal Ridge

In [48]:
#Instanciamos el modelo

lm_ridge =  linear_model.RidgeCV(alphas=[0.1, 20, 100], cv=5)

#Entrenamos el modelo
reg_lineal_ridge = lm_ridge.fit(X_train, y_train)

# Lo utilizamos para predecir en test
X_test = scaler_precio.transform(X_test)

#Guardamos el score en una variable
score_ridge_precio = reg_lineal_ridge.score(X_test, y_test)

#Prediccion con el set de testeo

print('Score del modelo Ridge:', reg_lineal_ridge.score(X_test, y_test))

Score del modelo Ridge: 0.5693368190898047


#### Como predice el mismo modelo el precio por metro cuadrado?

In [50]:
#Normalizamos las features

scaler_m2 = StandardScaler()
X_train_m2 = scaler_m2.fit_transform(X_train_m2)

In [51]:
print('media:',np.mean(X_train_m2[:,1]))
print('desvio:',np.std(X_train_m2[:,1]))

media: -2.3861648901743697e-17
desvio: 0.9999999999999999


In [60]:
#Instanciamos el modelo
lm_ridge_m2 =  linear_model.RidgeCV(alphas=[0.1, 20, 10], cv=5)

#Entrenamos el modelo
reg_lineal_ridge_m2 = lm_ridge_m2.fit(X_train_m2, y_train_m2)

# Lo utilizamos para predecir en test
X_test_m2 = scaler_m2.transform(X_test_m2)

score_ridge_m2 = reg_lineal_ridge_m2.score(X_test_m2, y_test_m2)

#Prediccion con el set de testeo
print('Score del modelo Ridge:', reg_lineal_ridge_m2.score(X_test_m2, y_test_m2))

Score del modelo Ridge: -5009344.4374822825


In [58]:
print('Score Ridge Precio: ', score_ridge_precio)
print('Alpha Ridge Precio: ', reg_lineal_ridge.alpha_)
print('Score Ridge m2: ', score_ridge_m2)
print('Alpha Ridge m2: ', reg_lineal_ridge_m2.alpha_)

Score Ridge Precio:  0.5693368190898047
Alpha Ridge Precio:  20
Score Ridge Precio:  0.22245239063011887
Alpha Ridge Precio:  20


## Regresion Lasso

In [62]:
#Instanciamos el modelo
lm_lasso =  linear_model.LassoCV(alphas=[1000, 100000, 1000], cv = 5)

#Entrenamos el modelo
reg_lineal_lasso = lm_lasso.fit(X_train, y_train)

#Guardamos el score
score_lasso_precio = reg_lineal_lasso.score(X_test, y_test)

print('Score del modelo Lasso:', reg_lineal_lasso.score(X_test, y_test))

Score del modelo Lasso: 0.5711304901647452


### Como funciona el modelo para el precio por metro cuadrado?

In [69]:
#Instanciamos el modelo
lm_lasso_m2 =  linear_model.LassoCV(alphas=[10000, 100000, 1000], cv = 5)

#Entrenamos el modelo
reg_lineal_lasso_m2 = lm_lasso_m2.fit(X_train_m2, y_train_m2)

#Guardamos el score
score_lasso_m2 = reg_lineal_lasso_m2.score(X_test_m2, y_test_m2)

print('Score del modelo Lasso:', reg_lineal_lasso_m2.score(X_test_m2, y_test_m2))

Score del modelo Lasso: -145.83933046587194


In [ ]:
print('Score Lasso Precio: ', score_lasso_precio)
print('Alpha Lasso Precio: ', reg_lineal_lasso.alpha_)
print('Score Lasso m2: ', score_lasso_m2)
print('Alpha Lasso m2: ', reg_lineal_lasso_m2.alpha_)

## Elastic Net

In [71]:
#Instanciamos el modelo

lm_elastic_net =  linear_model.ElasticNetCV(l1_ratio= [.1, .5, .7, .9, .95, .99, 1],cv = 5)

#Entrenamos el modelo
elastic_net = lm_elastic_net.fit(X_train, y_train)

#Guardamos el score
score_EN = elastic_net.score(X_test, y_test)

#Prediccion con el set de testeo

print('Score del modelo Elastic Net:', elastic_net.score(X_test, y_test))

Score del modelo Elastic Net: 0.5717193307475611


### Como funciona el modelo para el precio por metro cuadrado?

In [74]:
#Instanciamos el modelo

lm_elastic_net_m2 =  linear_model.ElasticNetCV(l1_ratio= [.1, .5, .7, .9, .95, .99, 1], alphas = [10, 100, 10],cv = 5)

#Entrenamos el modelo
elastic_net_m2 = lm_elastic_net_m2.fit(X_train_m2, y_train_m2)

#Guardamos el score
score_EN_m2 = elastic_net_m2.score(X_test_m2, y_test_m2)

#Prediccion con el set de testeo

print('Score del modelo Elastic Net:', elastic_net_m2.score(X_test_m2, y_test_m2))

Score del modelo Elastic Net: -3297167.2879575766


In [ ]:
print('Score Elastic Net Precio: ', score_EN)
print('Alpha Elastic Net Precio: ', elastic_net.alpha_)
print('l1_ratio Elastic Net Precio: ', elastic_net.l1_ratio_)
print('Score Elastic Net m2: ', score_EN_m2)
print('Alpha Elastic Net m2: ', elastic_net_m2.alpha_)
print('l1_ratio Elastic Net m2: ', elastic_net_m2.l1_ratio_)

# Modelo para Capital

In [77]:
df_cap = df.loc[df['Provincia']=='Capital Federal']


In [78]:
Cap_precio = df_cap.drop(['Precio', 'Precio_Dolares','Precio_m2_Dls', 'Precio_m2'], axis = 1)

Cap_m2 = df_cap.drop(['Precio', 'Precio_Dolares','Precio_m2_Dls', 'Precio_ARS'], axis = 1)

In [80]:
Cap_precio = Cap_precio.dropna(axis =0 , how = 'any')
Cap_m2 = Cap_m2.dropna(axis =0 , how = 'any')

In [81]:
#Creamos variables dummies de las variables categoricas

#Barrio
Cap_precio = pd.concat([Cap_precio, pd.get_dummies(data = Cap_precio['Barrio'], prefix= 'barrio')], axis =1)
Cap_m2 = pd.concat([Cap_m2, pd.get_dummies(data = Cap_m2['Barrio'], prefix= 'barrio')], axis = 1)

#Tipo Propiedad
Cap_precio = pd.concat([Cap_precio, pd.get_dummies(data = Cap_precio['Tipo_Propiedad'], prefix= 'tipo_prop')], axis =1)
Cap_m2 = pd.concat([Cap_m2, pd.get_dummies(data = Cap_m2['Tipo_Propiedad'], prefix= 'tipo_prop')], axis = 1)

In [82]:
#Dropeo de las columnas originales
Cap_precio = Cap_precio.drop(['Provincia','Barrio','Tipo_Propiedad'], axis = 1)
Cap_m2 = Cap_m2.drop(['Provincia','Barrio','Tipo_Propiedad'], axis = 1)

In [83]:
#Separamos nuestra matriz de features

X_Cap = Cap_precio.drop(['Precio_ARS'], axis = 1)
y_Cap = Cap_precio['Precio_ARS']

#### Separamos los datos en sets de testeo y training

In [84]:
#Separamos en train y test
X_train, X_test, y_train, y_test = train_test_split(X_Cap, y_Cap, test_size=0.10, random_state=1)

### Regresion Lineal

In [86]:
#Instanciamos el modelo

lm_Cap = linear_model.LinearRegression()

#Entrenamos el modelo
reg_lineal_Cap = lm_Cap.fit(X_train, y_train)

#Guardamos el score del modelo en una variable para poder comparar
R2_reg_lin_precio_Cap = reg_lineal_Cap.score(X_test, y_test)

#Prediccion con el set de testeo

print('Score del Regresion Lineal:', reg_lineal_Cap.score(X_test, y_test))

Score del Regresion Lineal: 0.5996776950274219


#### Como predice el mismo modelo el precio por metro cuadrado?

In [87]:
#Separamos nuestra matriz de features

X_Cap_m2 = Cap_m2.drop(['Precio_m2'], axis = 1)
y_Cap_m2 = Cap_m2['Precio_m2']

#Separamos en train y test
X_train_cap_m2, X_test_cap_m2, y_train_cap_m2, y_test_cap_m2 = train_test_split(X_Cap_m2, y_Cap_m2, test_size=0.10, random_state=0)

In [89]:
print(X_Cap_m2.shape, y_Cap_m2.shape)

(21418, 69) (21418,)


In [91]:
#Instanciamos el modelo

lm_cap_m2 = linear_model.LinearRegression()

#Entrenamos el modelo
reg_lineal_cap_m2 = lm_cap_m2.fit(X_train_cap_m2, y_train_cap_m2)

#Guardamos el score del modelo en una variable para poder comparar
R2_reg_lin_cap_m2 = reg_lineal_cap_m2.score(X_test_cap_m2, y_test_cap_m2)

#Prediccion con el set de testeo

print('Score del Regresion Lineal:', reg_lineal_cap_m2.score(X_test_cap_m2, y_test_cap_m2))

Score del Regresion Lineal: 0.05973424789070447
